In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('Tweets.csv')

In [3]:
data = data[['airline_sentiment','text']]

In [4]:
with open('twee','a',encoding = 'utf-8') as f:
    for string in data.text:
        f.writelines(string+'\n')

In [5]:
from gensim.models import word2vec

C:\Users\qiqi\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
sentences =word2vec.Text8Corpus("twee")  # 加载语料  
model =word2vec.Word2Vec(sentences, size=300)  #训练skip-gram模型，默认window=5 

In [7]:
word_vectors = model.wv
del model

In [8]:
data['vec'] = data.text.apply(lambda x :[word_vectors[w] for w in x.split() if w in word_vectors])

In [9]:
data = data[data['vec'].apply(lambda x:len(x)>5)]

In [10]:
del data['text']

In [11]:
data.airline_sentiment.unique()

array(['positive', 'neutral', 'negative'], dtype=object)

In [12]:
data.airline_sentiment.value_counts()

negative    9007
neutral     2789
positive    2013
Name: airline_sentiment, dtype: int64

In [13]:
dic = {'neutral':np.array([1,0,0]),'positive':np.array([0,1,0]),'negative':np.array([0,0,1])}

In [14]:
data['cat'] = data.airline_sentiment.map(dic)

In [15]:
del data['airline_sentiment']

In [16]:
data = data.reset_index()
del data['index']

In [17]:
maxlength = max([len(x) for x in data.vec])

In [18]:
maxlength

36

In [19]:
def pad(x):
    x1 = np.zeros((maxlength,300))
    x1[:len(x)] = x
    return x1

In [20]:
dataset = data.vec.apply(pad)

In [21]:
len(dataset),len(data.cat)

(13809, 13809)

In [22]:
labels = np.concatenate(data.cat).reshape(len(data.cat),-1)

In [23]:
np.shape(labels)

(13809, 3)

In [24]:
data_ = np.concatenate(dataset).reshape(len(dataset),maxlength,300)

In [25]:
np.shape(data_)

(13809, 36, 300)

打乱顺序选择作为训练和测试数据

In [26]:
index = np.random.permutation(int(len(data)))

In [27]:
label = labels[index]
dataset = data_[index]

In [28]:
label_train = label[:12000]
dataset_train = dataset[:12000]
label_test = label[12000:]
dataset_test = dataset[12000:]

# 搭建网络

In [29]:
import tensorflow as tf

In [30]:
learning_rate = 0.005
batch_size = 300
n_input = 300
n_steps = maxlength
n_hidden = 128
n_classes = 3

In [31]:
x = tf.placeholder(tf.float32, [None, n_steps,n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
output_keep_prob = tf.placeholder("float")

In [32]:
reg = tf.contrib.layers.l2_regularizer(scale=0.01)

In [33]:
def length(shuru):
    return tf.reduce_sum(tf.sign(tf.reduce_max(tf.abs(shuru),reduction_indices=2)),reduction_indices=1)

In [34]:
cell = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.GRUCell(n_hidden,
                kernel_initializer = tf.truncated_normal_initializer(stddev=0.0001),
                bias_initializer = tf.truncated_normal_initializer(stddev=0.0001)),
                output_keep_prob = output_keep_prob)

In [35]:
output, _ = tf.nn.dynamic_rnn(
            cell,
            x,
            dtype=tf.float32,
            sequence_length = length(x)
        )

In [36]:
output.get_shape()

TensorShape([Dimension(None), Dimension(36), Dimension(128)])

In [37]:
index = tf.range(0,batch_size)*n_steps + (tf.cast(length(x),tf.int32) - 1)
flat = tf.reshape(output,[-1,int(output.get_shape()[2])])
last = tf.gather(flat,index)

In [38]:
fc_1 = tf.contrib.layers.fully_connected(
                        last,
                        64,
                        weights_initializer=tf.truncated_normal_initializer(stddev=0.01),            
                        activation_fn = tf.nn.relu)
keep_prob = tf.placeholder("float")
fc1_drop = tf.nn.dropout(fc_1, keep_prob)

In [39]:
weight = tf.Variable(tf.truncated_normal([64, n_classes], stddev=0.001))
bias = tf.Variable(tf.constant(0.1, shape=[n_classes]))
prediction = tf.nn.softmax(tf.matmul(fc1_drop, weight) + bias)

In [40]:
cross_entropy = -tf.reduce_sum(y * tf.log(prediction))

In [41]:
weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

In [42]:
tf.contrib.layers.apply_regularization(reg,weights_list = weights)

<tf.Tensor 'get_regularization_penalty:0' shape=() dtype=float32>

In [43]:
reg_ws = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)

In [44]:
optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.9)
grads = optimizer.compute_gradients(cross_entropy + tf.reduce_sum(reg_ws))
for i, (g, v) in enumerate(grads):
    if g is not None:
        grads[i] = (tf.clip_by_norm(g, 5), v)  # clip gradients
train_op = optimizer.apply_gradients(grads)

C:\Users\qiqi\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [45]:
correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [46]:
def generatebatch(X,Y,n_examples, batch_size):
    for batch_i in range(n_examples // batch_size):
        start = batch_i*batch_size
        end = start + batch_size
        batch_xs = X[start:end]
        batch_ys = Y[start:end]
        yield batch_xs, batch_ys # 生成每一个batch

In [47]:
sess = tf.Session()

In [48]:
init = tf.global_variables_initializer() 
sess.run(init)

In [49]:
saver = tf.train.Saver()

In [50]:
for step in range(18):
    index_ = np.random.permutation(int(len(dataset_train)))
    dataset_train = dataset_train[index_]
    label_train = label_train[index_]
    for batch_x,batch_y in generatebatch(dataset_train,label_train,len(label_train),batch_size): 
        sess.run(train_op, feed_dict={x: batch_x, y: batch_y,keep_prob: 0.5,output_keep_prob:0.5})
    acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y,keep_prob: 1,output_keep_prob:1})
    loss = sess.run(cross_entropy, feed_dict={x: batch_x, y: batch_y,keep_prob: 1,output_keep_prob:1})
    saver.save(sess,'./lesson0',global_step = step)
    print("Iter " + str(step) + ", Minibatch Loss= " +  "{:.6f}".format(loss) + ", Training Accuracy= " +  "{:.5f}".format(acc))
print("Optimization Finished!")

Iter 0, Minibatch Loss= 198.027359, Training Accuracy= 0.76333
Iter 1, Minibatch Loss= 139.941513, Training Accuracy= 0.82333
Iter 2, Minibatch Loss= 142.179596, Training Accuracy= 0.82333
Iter 3, Minibatch Loss= 134.798965, Training Accuracy= 0.82333
Iter 4, Minibatch Loss= 105.706795, Training Accuracy= 0.85000
Iter 5, Minibatch Loss= 93.628601, Training Accuracy= 0.87667
Iter 6, Minibatch Loss= 52.342510, Training Accuracy= 0.95333
Iter 7, Minibatch Loss= 50.710182, Training Accuracy= 0.93000
Iter 8, Minibatch Loss= 33.270760, Training Accuracy= 0.96667
Iter 9, Minibatch Loss= 57.237392, Training Accuracy= 0.96000
Iter 10, Minibatch Loss= 28.329580, Training Accuracy= 0.97667
Iter 11, Minibatch Loss= 24.649929, Training Accuracy= 0.97667
Iter 12, Minibatch Loss= 26.550983, Training Accuracy= 0.96333
Iter 13, Minibatch Loss= 24.324686, Training Accuracy= 0.97667
Iter 14, Minibatch Loss= 19.379248, Training Accuracy= 0.98667
Iter 15, Minibatch Loss= 11.161270, Training Accuracy= 0.993

In [51]:
acc = []
for batch_x,batch_y in generatebatch(dataset_test,label_test,len(label_test),batch_size):
    accu = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y,keep_prob: 1,output_keep_prob:1})
    acc.append(accu)
acc = sum(acc)/len(acc)   
print(acc)

0.773333330949


In [79]:
import os
ckpt = tf.train.get_checkpoint_state(os.path.dirname('__file__'))
sess.run(tf.global_variables_initializer())
saver.restore(sess,ckpt.model_checkpoint_path)

INFO:tensorflow:Restoring parameters from lesson0-14


In [80]:
for step in range(30):
    index_ = np.random.permutation(int(len(dataset_train)))
    dataset_train = dataset_train[index_]
    label_train = label_train[index_]
    for batch_x,batch_y in generatebatch(dataset_train,label_train,7500,batch_size): 
        sess.run(train_op, feed_dict={x: batch_x, y: batch_y,keep_prob: 0.5,output_keep_prob:0.5})
    acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y,keep_prob: 1,output_keep_prob:1})
    loss = sess.run(cross_entropy, feed_dict={x: batch_x, y: batch_y,keep_prob: 1,output_keep_prob:1})
    saver.save(sess,'./lesson0',global_step = step)
    print("Iter " + str(step) + ", Minibatch Loss= " +  "{:.6f}".format(loss) + ", Training Accuracy= " +  "{:.5f}".format(acc))
print("Optimization Finished!")

Iter 0, Minibatch Loss= 37.266087, Training Accuracy= 0.87000
Iter 1, Minibatch Loss= 45.533707, Training Accuracy= 0.82000
Iter 2, Minibatch Loss= 52.623055, Training Accuracy= 0.77000
Iter 3, Minibatch Loss= 46.961140, Training Accuracy= 0.78000
Iter 4, Minibatch Loss= 34.564209, Training Accuracy= 0.89000
Iter 5, Minibatch Loss= 24.497801, Training Accuracy= 0.93000
Iter 6, Minibatch Loss= 21.895397, Training Accuracy= 0.89000
Iter 7, Minibatch Loss= 20.682911, Training Accuracy= 0.92000
Iter 8, Minibatch Loss= 11.348440, Training Accuracy= 0.97000
Iter 9, Minibatch Loss= 17.951738, Training Accuracy= 0.92000
Iter 10, Minibatch Loss= 11.972134, Training Accuracy= 0.93000
Iter 11, Minibatch Loss= 10.830638, Training Accuracy= 0.95000
Iter 12, Minibatch Loss= 8.233864, Training Accuracy= 0.97000
Iter 13, Minibatch Loss= 6.463031, Training Accuracy= 0.99000


KeyboardInterrupt: 

In [81]:
acc = []
for batch_x,batch_y in generatebatch(dataset_test,label_test,len(label_test),batch_size):
    accu = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y,keep_prob: 1,output_keep_prob:1})
    acc.append(accu)
acc = sum(acc)/len(acc)   
print(acc)

0.773333330949
